In [1]:
#%pip install transformers
#%pip install tensorflow
import tensorflow
from transformers import pipeline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
rev = pd.read_csv('../madrid/reviews_detailed/reviews_detailed.csv')

In [3]:
rev = rev.drop('reviewer_name', axis=1)

In [4]:
rev.dropna(axis=0, inplace=True)
rev.reset_index(drop=True, inplace=True)

In [5]:
rev.date=pd.to_datetime(rev.date, format='%Y-%m-%d')

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
rev.shape

(732410, 5)

In [8]:
from tqdm import tqdm

batch_size = 10000
batches = [rev[i:i+batch_size] for i in range(0, len(rev), batch_size)]

# Inicializar el diccionario para almacenar los resultados
edits = {}

# Procesar cada lote por separado
for batch in tqdm(batches, total=len(batches), leave=True):
    # Iterar sobre cada fila del lote
    for index, row in batch.iterrows():
        date = row["date"]
        comment = row["comments"]
        if pd.isnull(comment):  # Si el comentario es nulo, omitir la fila
            continue
        if not pd.isnull(date):
            date = pd.to_datetime(date).strftime("%Y-%m-%d")
        else:
            date = "No date available"
        comment_id = row["id"]
        if date not in edits:
            edits[date] = dict(sentiments=list(), edit_count=0, comment_ids=list())
        edits[date]["edit_count"] += 1
        edits[date]["comment_ids"].append(comment_id)
        edits[date]["sentiments"].append(find_sentiment(comment))

# Combinar los resultados en un DataFrame
results = pd.concat([pd.DataFrame({"date": k, **v}) for k, v in edits.items()], ignore_index=True)

  0%|          | 0/74 [05:01<?, ?it/s]


KeyboardInterrupt: 